In [1]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from function import baseball_lambda

/Users/rosclay/Documents/VSCode/langgraph-multi-agent/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
Python REPL can execute arbitrary code. Use with caution.


In [2]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [3]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs - {today}"

In [4]:
session_id = '2110956'

In [5]:
task =  "Consider the first week of August 2020 - find the names of 3 pitchers who's curveballs were most similar to Max Scherzer's."
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Retrieve Node ***

Distance to neareast plan: 0.6294779777526855

*** Entered Initialize Node ***

Formulating a new plan to solve the task.

*** Entered Update Node ***

Updating plan based function documentation
To find the 3 pitchers with curveballs most similar to Max Scherzer's in the first week of August 2020:

1. Use playerid_lookup to find Max Scherzer's MLBAM player ID by passing 'scherzer' as the last name and 'max' as the first name
2. Get Max Scherzer's statcast pitching data for 2020-08-01 to 2020-08-07 using statcast_pitcher, passing '2020-08-01' for start_dt, '2020-08-07' for end_dt, and Scherzer's MLBAM ID for player_id 
3. Filter Scherzer's data to only curveballs using the pitch_type field
4. Get statcast pitching data for all pitchers from 2020-08-01 to 2020-08-07 using statcast_pitcher, passing '2020-08-01' for start_dt and '2020-08-07' for end_dt, but omitting player_id to get all pitchers
5. Filter the all pitcher dat

In [6]:
task = """
make sure the plan follows this general flow:
1) create an average vector for each pitcher's curveball.
2) Train a knn model on this data
3) Use this model to find the 3 pitchers with the most similar curveball to Max Scherzer
"""
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Revise Node ***

Here is the revised plan to find the 3 pitchers with curveballs most similar to Max Scherzer's in the first week of August 2020:

1. Use playerid_lookup to find Max Scherzer's MLBAM player ID by passing 'scherzer' as the last name and 'max' as the first name

2. Get statcast pitching data for all pitchers from 2020-08-01 to 2020-08-07 using statcast_pitcher, passing '2020-08-01' for start_dt and '2020-08-07' for end_dt, but omitting player_id to get all pitchers

3. Filter the data to only include curveballs using the pitch_type field

4. Group the data by pitcher and calculate the average value of each relevant statcast metric for each pitcher's curveball. This will give us an average curveball vector for each pitcher.

5. Separate Max Scherzer's average curveball vector from the other pitchers. 

6. Use the average curveball vectors (excluding Scherzer) to train a KNN model. 

7. Use the trained KNN model with Scherzer's

In [7]:
task = 'approved'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***
Unknown plan. Executing with Opus

*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
from pybaseball import playerid_lookup

# 1. Use playerid_lookup to find Max Scherzer's MLBAM player ID 
scherzer_id_results = playerid_lookup('scherzer', 'max')
scherzer_id = scherzer_id_results.iloc[0].key_mlbam

print(f"Max Scherzer's MLBAM ID is: {scherzer_id}")

Result:
Gathering player lookup table. This may take a moment.
Max Scherzer's MLBAM ID is: 453286


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
from pybaseball import statcast_pitcher

# 2. Get statcast pitching data for all pitchers from 2020-08-01 to 2020-08-07
data = statcast_pitcher('2020-08-01', '2020-08-07')

print(f"{len(data)} statcast pitching records found from 2020-08-01 to 2020-08-07")

Result:
ValueError("Player ID is required. If you need to find a player's id, try pybaseball.playerid_lookup(last_name, first_name) and use their key_mlbam. 

  0%|          | 0/7 [00:00<?, ?it/s]


*** Entered Execute Node ***


Executing:
from pybaseball import statcast

# 2. Get statcast pitching data for all pitchers from 2020-08-01 to 2020-08-07 
data = statcast('2020-08-01', '2020-08-07')

print(f"{len(data)} statcast records found from 2020-08-01 to 2020-08-07")


100%|██████████| 7/7 [00:11<00:00,  1.69s/it]



Result:
This is a large query, it may take a moment to complete
24877 statcast records found from 2020-08-01 to 2020-08-07


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
# 3. Filter the data to only include curveballs
print(data.pitch_type.unique())

Result:
['FC' 'KC' 'FF' 'FS' 'SL' 'SI' 'CU' 'CH' 'ST' 'PO' 'FA' 'CS' 'SV']


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
# 3. Filter the data to only include curveballs
curveballs = data[data.pitch_type == 'CU']

print(f"There are {len(curveballs)} curveball pitches in the dataset")

Result:
There are 2025 curveball pitches in the dataset


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
import numpy as np

# 4. Group by pitcher and calculate average curveball metrics 
pitcher_avgs = curveballs.groupby('pitcher')[['release_speed', 'release_spin_rate', 
                                              'pfx_x', 'pfx_z', 
                                       

In [8]:
task = 'yes'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Memorize Node ***

Thank you, task has been commited to memory
